# Technical Assessment

### Import packages, upload files into notebook

In [1]:
#import packages
from google.colab import files
from datetime import datetime

import sqlite3
import pandas as pd
import numpy as np



In [2]:
#Use this cell to upload the excel file for this analysis
uploaded = files.upload()

Saving AssessmentData.xlsx to AssessmentData (1).xlsx


#Problem 2 - SQL Portion

In [3]:
filename = 'AssessmentData.xlsx'

## Load Data

In [4]:
#Load Problem 2 Data into dataframes
Timecard_Data = pd.read_excel(filename, sheet_name='Timecard_Data')
User_Data = pd.read_excel(filename, sheet_name = 'User_Data')
Sales_Data = pd.read_excel(filename, sheet_name = 'Sales_Data')
Project_Data = pd.read_excel(filename, sheet_name = 'Project_Data')

## Connect to SQLite Database to act as SQL database

In [5]:
#Create a connection to a SQLlite database
conn = sqlite3.connect('sample.db')
c = conn.cursor()

In [6]:
#Load the Timecard_Data, User_Data, Sales_Data, and Project_Data dataframes into the SQLite database
Timecard_Data.to_sql('Timecard_Data', conn, if_exists='replace', index=False)
User_Data.to_sql('User_Data', conn, if_exists='replace', index=False)
Sales_Data.to_sql('Sales_Data', conn, if_exists='replace', index=False)
Project_Data.to_sql('Project_Data', conn, if_exists='replace', index=False)

2349

In [7]:
#Query 1: Return all records from the User_Data table where the user was active, but did not log any time in the month of December, 2020.

query1 = '''
SELECT * FROM User_Data
WHERE User_Start_Date <= '2020-12-31'
AND User_End_Date >= '2020-12-1'
AND User_ID NOT IN (
SELECT User_ID FROM Timecard_Data
WHERE Timecard_Date >= '2020-12-1' AND Timecard_Date <= '2020-12-31'
)
'''

result1 = pd.read_sql(query1, conn)
print(result1)


    User_ID                          User_Role User_Geography User_Team  \
0  12263685  Research Services Project Manager           AMER     PM-NA   
1  12263895  Research Services Project Manager           AMER     PM-NA   

         User_End_Date      User_Start_Date  
0  2021-01-12 00:00:00  2019-09-20 00:00:00  
1  2020-12-10 00:00:00  2019-09-20 00:00:00  


In [8]:
#Query 2: Return a table that sums the Services_Budget for each Workspace_Region

query2 = '''
SELECT SUM(Services_Budget), Workspace_Region
FROM Sales_Data
INNER JOIN Project_Data ON Project_Data.Opportunity_ID = Sales_Data.Opportunity_ID
GROUP BY Workspace_Region
'''

result2 = pd.read_sql(query2, conn)
print(result2)

   SUM(Services_Budget) Workspace_Region
0            5867444.49             APAC
1            9289837.51             EMEA
2            1410834.22    Latin America
3           60164150.09    North America


In [9]:
#Query 3: Return a table that counts the number of users that have booked time to more than one Workspace_Region in 2021

query3 = '''
SELECT COUNT(*) AS multiple_workspace_count
FROM
(
SELECT User_ID, COUNT(DISTINCT(Workspace_Region)) as num_regions
FROM Timecard_Data
LEFT JOIN Project_Data ON Project_Data.Workspace_ID = Timecard_Data.Workspace_ID
WHERE Timecard_Date >= '2021-01-01'
AND Timecard_Date <= '2021-12-31'
GROUP BY User_Id
HAVING COUNT(DISTINCT(Workspace_Region)) > 1
)
AS subquery
'''

result3 = pd.read_sql(query3, conn)
print(result3)

   multiple_workspace_count
0                        44


In [10]:
#This code block was used for data validation
query4 = '''
SELECT SUM(td.Hours_Logged)
FROM Timecard_Data td
JOIN Project_Data pd ON td.Workspace_ID = pd.Workspace_ID
WHERE strftime('%Y-%m', td.Timecard_Date) = '2020-12'
  AND pd.Project_Type = 'Billable';
'''

In [11]:
#This code block is used to run the SQL queries
result = pd.read_sql(query4, conn)
print(result)

   SUM(td.Hours_Logged)
0           8875.733333


# Problem 2 - Analysis Portion
Processing the User_Data table for further analysis in Tableau

In [12]:
#We're going to be modifying a copy of our User_Data table.  First create a copy.
User_Data_Hours = User_Data.copy(deep=True)

In [13]:
#Display first few rows for reference
display(User_Data_Hours.head())

,User_ID,User_Role,User_Geography,User_Team,User_End_Date,User_Start_Date
0,9642765,Service Delivery Manager,AMER,PM-NA,NaT,2018-02-15
1,9805405,RX Analyst,AMER,RX-NA-RA,2021-01-12,2018-03-22
2,9862085,Research Services Project Manager,AMER,PM-NA,NaT,2018-04-12
3,9909425,Research Services Project Manager,AMER,PM-NA,NaT,2018-04-18
4,9909435,RX Analyst,AMER,RX-NA-RA,NaT,2018-04-18


In [14]:
#Confirm datatypes that we'll be working with
User_Data_Hours.dtypes

User_ID                     int64
User_Role                  object
User_Geography             object
User_Team                  object
User_End_Date      datetime64[ns]
User_Start_Date    datetime64[ns]
dtype: object

In [15]:
#First find out how many null values are in these columns
print(User_Data_Hours["User_End_Date"].isnull().sum())
print(User_Data_Hours["User_Start_Date"].isnull().sum())

124
0


In [16]:
#Check if there are any duplicate User_ID
duplicate_user_ID = User_Data["User_ID"].duplicated()
print(set(duplicate_user_ID))

{False}


In [17]:
#For the purpose of this calculation, let's replace User_End_Date with the most recent date of activity.  We pull this date from the Timecard_Data table.
most_recent_activity = max(Timecard_Data["Timecard_Date"])
print(most_recent_activity)

2021-03-14 00:00:00


In [18]:
#We need to make sure there are no null values in the end date field.  For the purpose of calculation, we assign our most recent date of activity (calculated in the block above) to User_End_Date.

#Use a boolean mask to identify null values and replace them with our most_recent_activity
#create mask
mask = User_Data_Hours["User_End_Date"].isnull()

#apply mask to modify dataframe
User_Data_Hours.loc[mask, "User_End_Date"] = most_recent_activity

In [19]:
#confirm it is working properly.
display(User_Data_Hours.head())
print("Count of Null Values in User_End_Date:",User_Data_Hours["User_End_Date"].isnull().sum())

,User_ID,User_Role,User_Geography,User_Team,User_End_Date,User_Start_Date
0,9642765,Service Delivery Manager,AMER,PM-NA,2021-03-14,2018-02-15
1,9805405,RX Analyst,AMER,RX-NA-RA,2021-01-12,2018-03-22
2,9862085,Research Services Project Manager,AMER,PM-NA,2021-03-14,2018-04-12
3,9909425,Research Services Project Manager,AMER,PM-NA,2021-03-14,2018-04-18
4,9909435,RX Analyst,AMER,RX-NA-RA,2021-03-14,2018-04-18


Count of Null Values in User_End_Date: 0


In [20]:
print(User_Data_Hours.dtypes)

User_ID                     int64
User_Role                  object
User_Geography             object
User_Team                  object
User_End_Date      datetime64[ns]
User_Start_Date    datetime64[ns]
dtype: object


In [21]:
#The "busday" function in Numpy calculates the number of non-weekend days.  Below we apply it to our dataframe to create the new "Workdays" column

User_Data_Hours["Workdays"] = np.busday_count(User_Data_Hours["User_Start_Date"].values.astype('datetime64[D]'),
                                 User_Data_Hours["User_End_Date"].values.astype('datetime64[D]'))

In [22]:
#Check our results.  Spot checked using timeanddate.com to confirm accuracy
display(User_Data_Hours.head())

,User_ID,User_Role,User_Geography,User_Team,User_End_Date,User_Start_Date,Workdays
0,9642765,Service Delivery Manager,AMER,PM-NA,2021-03-14,2018-02-15,802
1,9805405,RX Analyst,AMER,RX-NA-RA,2021-01-12,2018-03-22,733
2,9862085,Research Services Project Manager,AMER,PM-NA,2021-03-14,2018-04-12,762
3,9909425,Research Services Project Manager,AMER,PM-NA,2021-03-14,2018-04-18,758
4,9909435,RX Analyst,AMER,RX-NA-RA,2021-03-14,2018-04-18,758


In [23]:
# prompt: Output my User_Data_hours file as an excel file.

#User_Data_Hours.to_excel('User_Data_Hours.xlsx', index=False)
#files.download('User_Data_Hours.xlsx')
